# Notes on the variables

## Introduction
In this notebook we did research on the features of the data set and had a look at the data preprocessing necessary for our project. Based on this research, we made decisions concerning outlier treatment, imputation of missing values, binning, one-hot encoding, and scaling. Let's first understand the reason to use these processing measures.

**Outlier treatment:**
Data can have mistakes, deriving for multiple reasons. These outliers can have undesirable effects when analysing the data. As these outliers are mostly the result of (human) errors, we want to take these out. Therefore we apply an outlier treatment, namely capping our data set per feature. The cap is set at 3 standard deviations from the mean for every continuous variable (3 standard deviations below and above the mean).

**Imputation of missing values:**
Also known as gap-filling, this methods concerns artificially completing a data set by imputing missing values. Sometimes information is missing from certain instances of a data set. If this information is not all important, we don't simply want to remove the whole instance as there is still information available. What we did for missing values is to make an intelligent guess of the missing information. We decided on a method per feature.

**Binning:**
Binning is important for two reasons:
* it makes computations a lot faster.
* on a conceptual level binning categorizes a continuous variable. This makes it easy for the model to understand that variable. For example, we frequently do binning without realizing: when somebody says an album was released 1967 we automatically think it was released in the 60's and that in itself is a valuable piece of information.

Binning was done for all continuous variables.

**One-hot encoding:**
Also known as label encoding, this method solves the issue of categorical values. Mostly, categorical values do not have a natural distance assigned. As most machine learning methods work with distances (less/greater or equal than), this is an obstacle. By one-hot-encoding, we solve this issue.

**Scaling:**
Some of our machine learning algorithms are based on finding the most 'similar' data point, where similaraity is measured by distances. If the domain of a certain feature is of a different scale than another, the impact of this feature on the distances between the data point will differ as well. For example, suppose we're looking at the weight in kg and the length in m of people. A difference of 0.30 kg we would consider as a minor difference, whereas a difference of 0.30 m, would be considered as a major difference. Still the impact on the distance between two points would be treated the same. Therefore we want to standardise the feature domains.

The rest of this notebook will be take the reader through our findings and methods of preprocessing

## Basic EDA

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.stats import iqr
%pprint #disables pretty printing to see printed lists horizontally
%matplotlib inline
pd.set_option("display.precision",2)
pd.set_option("display.max_rows",30)
pd.set_option("display.max_columns",40)

In [ ]:
def df_reset():
    global df,target,target_donation_amount
    raw_df = pd.read_csv("Donors_dataset.csv")
    df = raw_df.drop(["FILE_AVG_GIFT","CONTROL_NUMBER"],axis = 1).copy()
    ordered_names = list(df.columns)
    ordered_names.sort()
    df = df[ordered_names]
    #column names to lowercase
    df.columns = df.columns.str.lower()
    df.reset_index(inplace=True)
    

In [ ]:
df_reset()
raw_df = pd.read_csv("Donors_dataset.csv")
df.describe()

**LABEL ENCODING:** Used for exploring correlations

In [ ]:
def label_encode(df):
    df_reset()
    df.recency_status_96nk = df.recency_status_96nk.map( {'E':0,'L':1,'F':2, 'N':3, 'A':4, 'S':5} ).astype('int64')
    df.frequency_status_97nk = df.frequency_status_97nk.map( {4:'A',3:'B',2:'C', 1:'D'} )
    df['donor_gender'].values[df.donor_gender == 'A'] = 'F'
    df.donor_gender = df.donor_gender.map( {'F': 1, 'M': 0,'U':3} ).astype('int64')
    df.home_owner = df.home_owner.map( {'H': 1, 'U': 0} ).astype('int64')
    df.overlay_source = df.overlay_source.map({'P':1, 'B':2, 'N':3, 'M':4})
    df.urbanicity = df.urbanicity.map({'U':1,'C':2,'S':3,'T':4,'R':5,'?':6})
    df['ses'].values[df.ses == '?'] = 5
    df.ses = df.ses.astype('int64')
    return df
df_label_encoded =label_encode(df)

A matrix with correlations

In [ ]:
sorted_df = raw_df.reindex(sorted(raw_df.columns), axis=1)
corrmat = sorted_df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=1., vmin=-.5, square=True);

let's check the data types

In [ ]:
types = df.dtypes
types

#### Cluster_code analysis

In [ ]:
unique_vals = df.cluster_code.unique()
sorted(unique_vals)

there's a messy '.' that should be addressed, so let's investigate

In [ ]:
df[(df.cluster_code =='.')].shape

This variable highly correlates with urbanicity and ses values. Let's look into that. First let's look at the unique values of cluster code for each urbanicity value.

In [ ]:
urban_values = ['U','C','S','T','R','?']
urb_cluster = df[['urbanicity', 'cluster_code']]
grouped = urb_cluster.groupby('urbanicity').groups
print('Urbanicity   :   Unique cluster values')
for key in urban_values:
    print(key, '   :   ', sorted(urb_cluster.cluster_code.iloc[grouped[key]].unique()))


clearly, there is straighforward connection between cluster_code and urbanicity. Let's how it relates to ses.

In [ ]:
ses_values = ['1', '2' , '3', '4','?']
ses_cluster = df[['ses', 'cluster_code']]
grouped = ses_cluster.groupby('ses').groups
for key in ses_values:
    print(key, ':', sorted(ses_cluster.cluster_code.iloc[grouped[key]].unique()))


The relationship between these two values is not so obvious as with urbanicity. Still, each cluster_code value only corresponds to one value of ses. Also the values of '.' and '?' correspond to missing data in all three fields. This indicates that there might be some problem with those data points so they should not be imputed but given their own value. Even so, both approaches were tried and there was not a significant change in the results. One thing that should be noted is that urbanicity is then the corresponding of a binned version of cluster code, however the same doesn't hold for ses. Also, since there are many of cluster_code for each value of ses/urbanicity, cluster code probably has more information. 


In [ ]:
weird_cluster_code_date = df[df.cluster_code == '.']
print(weird_cluster_code_date)

We finally decided to take out all the elements that have cluster code '.' as these points have weird data all over.

Furthermore, no data preprocessing was necessary.

#### Donor gender

In [ ]:
df.donor_gender.value_counts()

the values A and U are problematic. We impute A with F, and make U into it's own class of number 3.

In [ ]:
#Imputation
df['donor_gender'].values[df.donor_gender == 'A'] = 'F'

Let's check everything is ok

In [ ]:
df.donor_gender.value_counts()

now let's put the gender into a binary variable

In [ ]:
df.donor_gender = df.donor_gender.map( {'F': 1, 'M': 0, 'U': 3} ).astype('int64')

let's check what are the most correlated variables correlated

In [ ]:
df_label_encoded.corr().abs()['donor_gender'].sort_values(ascending=False)[:10]

In [ ]:
df.home_owner = df.home_owner.map( {'H': 1, 'U': 0} ).astype('int64')

In [ ]:
sns.barplot(data= df,x='donor_gender',y='home_owner').set_title('Fraction of home_owners by donor gender')

There is a very low correlation with all the variables and it is barely noticeable in the graph. Probably best to leave the U values as they are. Even so, an approach where the U values were imputed with the mode was also tried but it didn't yield better results

#### Donor age

In [ ]:
df_reset()
df.donor_age.isnull()

Donor age has many missing values, we'll start out by imputing with the average, but first let's take a look at the variable

In [ ]:
df.donor_age.fillna(value=df.donor_age.mean(),inplace = True)

there are some outliers that should be analyzed

In [ ]:
#sorted(df.donor_age)[:100]

probably there are no donors aged less than 5 years old and it is a mistake (pssibly even more, but since it will be binned it doesn't matter). For now, let's replace the values with the mean (should be mean of donors with age above 5 years old. Otherwise you include the "wrong" ages!)

In [ ]:
df_label_encoded.corr().abs()['donor_age'].sort_values(ascending=False)[:15]

In [ ]:
pd.cut(df.donor_age,bins=[0,20,30,40,50,70,90])

In [ ]:
df['age_bin'] = pd.cut(df.donor_age,bins=[0,20,30,40,50,70,90])

In [ ]:
sns.barplot(data=df,x="age_bin",y="months_since_origin")

In [ ]:
df.age_bin[df.age_bin==0]

In [ ]:
df['age_bin'] = pd.cut(df.donor_age,bins=[0,20,30,40,50,70,80,90])

In [ ]:
sns.barplot(data=df, x="age_bin",y="months_since_origin")

this variable is nicely correlated with donor_age which means that it can be used in order to impute donor_age. However, there are a few problems with this relationship. First, the fact that there are  so many values bellow 20 in donor_age. In fact, upon further investigation:

In [ ]:
list(df.donor_age.sort_values()[:100])

These ages are surely a mistake. Let's look at what the sorted values of months_since_origin are for this variable.

In [ ]:
df[['donor_age','months_since_origin']].sort_values(by=['donor_age','months_since_origin'],ascending =[True,False])[:300]

There are a couple of things to note. The first one is that these low age entries actually have the highest values of months_since_origin in the whole dataset. Given the chart presented above and the clear correlation between the two variables this might indicate that perhaps the mistake is that perhaps the system was not prepared to receive values higher than 100 in the age field so age=0 might actually correspond to 100. Also the maximum value for the age is only 87. However, there are two other points that go against this theory. The first would be that the high months_since_origin values stop abruptly at the age of 18 and the other one being that we have less 0 age values than 2, and less 2 values than 6 and so on. If that number indicated the age in the hundreds, it would be the other way around. Given all this weirdness, the final decision was to transform all the ages bellow 18 to nan and impute everything according to the average donor age value by months_since_origin. 

This is not such an easy task as it may seem. Initially, it was planned to impute the nan values according to the the mean for a given 'months_since_origin' value. This would be that table:

In [ ]:
impute_table = df[['months_since_origin','donor_age']][df.donor_age.notnull()].groupby(['months_since_origin']).mean()
impute_table

The problem is that there are values of months_since_origin that only have nan in the donor_age field, so it's not possible to have a value of reference for that value:

In [ ]:
nan_unique_vals = []
for i in df.months_since_origin[df.donor_age.isnull()].unique():
    if i not in impute_table.index:
        nan_unique_vals.append(i)

print ('list with values of months_since origin that don\'t have any age value:',nan_unique_vals)

Since the relationship between these variables seem to be linear (excluding those with less than 18 years old), it was decided to impute the above mentioned with a linear regression model.

In [ ]:
from sklearn.linear_model import LinearRegression
df.loc[df.donor_age <=18,'donor_age']=np.NaN
X = df.months_since_origin[df.donor_age.notnull()].values.reshape(-1,1)
y = df.donor_age[df.donor_age.notnull()].values.reshape(-1,1)
impute_fit=LinearRegression()
impute_fit = impute_fit.fit(X,y)

In [ ]:
for nan_age_index in df.donor_age[df.donor_age.isnull()].index:
    if df.months_since_origin.iloc[nan_age_index] in nan_unique_vals:
        #impute with model in case there is no base reference to impute
        df.at[nan_age_index,'donor_age'] = impute_fit.predict(df.months_since_origin.loc[nan_age_index].reshape(1,-1)).flatten()
    else:
        #impute according to impute_table otherwise
        df.at[nan_age_index,'donor_age'] = impute_table.loc[df.months_since_origin.iloc[nan_age_index]]


#### File card gift

In [ ]:
df.file_card_gift.unique()

In [ ]:
sns.displot(df.file_card_gift)

everything seems in order, let's just convert to float as it is a dollar amount (and other dollar amounts are also in floats)

In [ ]:
df.file_card_gift = df.file_card_gift.astype("float64")

#### FREQUENCY_STATUS_97NK

In [ ]:
df.frequency_status_97nk.unique()

One-hot encoding was tested but it yielded worse results

#### Home owner

In [ ]:
df.home_owner.value_counts()

there are many unknown values. Let's just turn this into a binary variable for now.

In [ ]:
df.home_owner = df.home_owner.map( {'H': 1, 'U': 0} ).astype('int64')

#### Income group

In [ ]:
df.income_group.unique()

In [ ]:
df.income_group

this should be a categorical variable (int) but it's in float

In [ ]:
sns.displot(df.income_group,kde=True,)

In [ ]:
df.income_group.isnull().sum()

there are many null values. Let's create a different category (value 0) for them for now

In [ ]:
df.income_group = df.income_group.fillna(value=0.)

In [ ]:
df.income_group = df.income_group.astype('int64')
df.income_group.dtype

this column should be a normal distribution even though it doesn't look a lot like one

#### 2nd iteration

In [ ]:
df_reset()

Let's check correlations

In [ ]:
df_label_encoded.corr().abs()['income_group'].sort_values(ascending=False)[:15]

Let's check if the median_household_income values corresponding to nan in income group are of good quality

In [ ]:
df[['income_group','median_household_income']][df.income_group.isnull()]

they seem like they are. Let's visualize the relationship between these two variables

In [ ]:
sns.barplot(data=df,x="income_group",y="median_household_income")

It seems like it's perfecto to serve as basis for imputation. Just note that we should impute median_household_income first since there are a handfull of zero values that should be filled first. Also note that the mean for imputation is rounded.

In [ ]:
#imputing median_household_income first
#set the 0 to nan
df.loc[df.median_household_income == 0,'median_household_income']=np.NaN
#group by urbanicity and ses, calculate average and use that
df['median_household_income'] = df.groupby(['ses','urbanicity'], sort=False)['median_household_income'].apply(lambda x: x.fillna(x.mean()))

Also, since this a continuous variable, it should be binned.

In [ ]:
#binning
df['median_household_bin'] = pd.qcut(df.median_household_income,7,labels=False)
#imputation
df['income_group'] = df.groupby(['median_household_bin'], sort=False)['income_group'].apply(lambda x: x.fillna(x.mean())).copy()
df['income_group'] = df['income_group'].round()
df.drop('median_household_bin',axis=1,inplace=True)

also this variable should be one-hot encoded, to let's convert it to string

In [ ]:
df['income_group'] = df['income_group'].astype('str')
df['income_group'].unique()

#### In house

In [ ]:
df.in_house.sum()

### Last gift amount AND Lifetime* variables

this section includes 
* **LAST_GIFT_AMT** - amount of the most recent donation from the individual to the charitable organization
* **LIFETIME_AVG_GIFT_AMT** - lifetime average donation (in \\$) from the individual to the charitable organization
* **LIFETIME_CARD_PROM** - total number of card promotions sent to the individual by the charitable organization
* **LIFETIME_GIFT_AMOUNT** - total lifetime donation amount (in \\$) from the individual to the charitable organization
* **LIFETIME_GIFT_COUNT** - total number of donations from the individual to the charitable organization
* **LIFETIME_GIFT_RANGE** - maximum donation amount from the individual minus minimum donation amount from the individual
* **LIFETIME_MAX_GIFT_AMT** - maximum donation amount (in \\$) from the individual to the charitable organization
* **LIFETIME_MIN_GIFT_AMT** - minimum donation amount (in \\$) from the individual to the charitable organization
* **LIFETIME_PROM** - total number of promotions sent to the individual by the charitable organization

#### Binning and outlier treatment

To guarentee that the binning makes sense, a threshold should be employed so that most bins aren't empty because of the outliers. Let's look at the Last_gift_amt column as an example. The distribution goes like this

In [ ]:
sns.displot(df.last_gift_amt,kde=True,)

For all the above features, outlier treatment and binning was applied. For outlier treatment we decided upon thresholds of 3 standard deviations above and below the average of the feature. For the above example that would be:

In [ ]:
upper_threshold = df['last_gift_amt'].mean()+3*df['last_gift_amt'].std()
print(upper_threshold)

Which makes a lot of sense!

<b>MEDIAN HOME VALUE</b> - median home value (in 100$) as determined by other input variables

In [ ]:
df_reset()

In [ ]:
df.median_home_value.isnull().sum()

In [ ]:
sns.displot(df.median_home_value)

In [ ]:
df.median_home_value.value_counts()

Let's explore the connections with urbanicity and SES

In [ ]:
order = ['U','C','S','T','R','?']
fig, axs = plt.subplots(ncols=2)
sns.countplot(x=df["urbanicity"],hue_order=order, order =order, ax = axs[0]).set_title('Urbanicity for whole dataset')
sns.countplot(x=df[df.median_home_value ==0]["urbanicity"],hue_order=order, order =order, ax = axs[1]).set_title('Urbanicity for MHV=0')
plt.tight_layout()

In [ ]:
df.ses.unique()

In [ ]:
order = ['?', '2', '1', '3', '4']
fig, axs = plt.subplots(ncols=2)
sns.countplot(x=df["ses"],hue_order=order, order =order, ax = axs[0]).set_title('SES for whole dataset')
sns.countplot(x=df[df.median_home_value ==0]["ses"],hue_order=order, order =order, ax = axs[1]).set_title('SES for MHV=0')
plt.tight_layout()

it seems like these values tend to be more urban and in the case of ses the distribution of values is clearly skewed to the right comparing with all the data, but that doesn't tell us too much. Let's check if there is a correlation with age

In [ ]:
sns.kdeplot(df[df.median_home_value==0]['donor_age'])
sns.kdeplot(df['donor_age'])
plt.legend(labels=['MHV = 0', 'all data'])

it seems like these are younger donors. Let's what are the variables that correlate the most with median_home_value in order to impute according to that

In [ ]:
df_label_encoded.corr().abs()['median_home_value'].sort_values(ascending=False)[:10]

per_capita_income could be a good candidate. Let's check if it has integrity enough to base our imputation.

In [ ]:
df[df.median_home_value==0]['per_capita_income']

most values are zero, so not good enough. That could be a reason in fact why there is a high correlation between these is so high. Let's check the next candidate

In [ ]:
df[df.median_home_value==0]['median_household_income']

the fact that almost everything is zero is suspicious. Let's count some of these values

In [ ]:
df[(df.median_household_income==0) & (df.per_capita_income==0)].shape[0]

In [ ]:
df[df.median_home_value==0].shape[0]

In [ ]:
df[df.per_capita_income==0].shape[0]

there are 173 data points that have 0 on these three fields. This will help us treating these variables.

income_group also has many Nans. Imputing based on urbanicity and ses seems like a good strategy since earlier it was seen that there is a relationship between these points that that value. Also it makes sense intuitively that house value is related with socioeconomic status.

In [ ]:
#set the 0 to nan
df.loc[df.median_home_value == 0,'median_home_value']=np.NaN
#group by urbanicity and ses, calculate average and use that
df['median_home_value'] = df.groupby(['ses','urbanicity'], sort=False)['median_home_value'].apply(lambda x: x.fillna(x.mean()))

<b>MEDIAN HOUSEHOLD INCOME</b> - median household income (in 100$) as determined by other input variables

In [ ]:
sns.displot(df.median_household_income)

There is a weird peak at 0. Instead of filling zero values with the mean we follow the same strategy as before

In [ ]:
#set the 0 to nan
df.loc[df.median_household_income == 0,'median_household_income']=np.NaN
#group by urbanicity and ses, calculate average and use that
df['median_household_income'] = df.groupby(['ses','urbanicity'], sort=False)['median_household_income'].apply(lambda x: x.fillna(x.mean()))

<b>MONTHS SINCE FIRST GIFT</b> - number of months since the first donation from the individual to the charitable organization

In [ ]:
df_reset()
df.months_since_first_gift.plot(kind = 'hist')

Should be outlier corrected!

<b>MONTHS SINCE LAST GIFT</b> - number of months since the most recent donation from the individual to the charitable organization

In [ ]:
df.months_since_last_gift.plot(kind = 'hist')

In [ ]:
df.months_since_first_gift.describe()

Should be outlier corrected!


<b>MONTHS SINCE LAST PROM RESP</b> - number of months since the individual has responded to a promotion by the charitable organization

In [ ]:
df.months_since_last_prom_resp.plot(kind = 'hist')

In [ ]:
df.months_since_last_prom_resp.describe()

Currently a float but should be int - checking why

In [ ]:
df.months_since_last_prom_resp.dtype

In [ ]:
sorted(df.months_since_last_prom_resp)[:10]

There are negative values which is weird (cannot have negative time) - changing these to 0.

In [ ]:
df['months_since_last_prom_resp'].values[df.months_since_last_prom_resp<1] = 0

In [ ]:
sorted(df.months_since_last_prom_resp)[:10]

Also checking for null values

In [ ]:
df.months_since_last_prom_resp.isnull().sum()

246 null values - changing these for the average

In [ ]:
df.months_since_last_prom_resp.fillna(value=df.months_since_last_prom_resp.mean(),inplace = True)

Now changing from float to integer

In [ ]:
df.months_since_last_prom_resp = df.months_since_last_prom_resp.astype('int64')
df.months_since_last_prom_resp.dtype

<b>MONTHS SINCE ORIGIN</b> - number of months that the individual has been in the charitable organization's database

In [ ]:
df.months_since_origin.plot(kind = 'hist')

In [ ]:
df.months_since_origin.isnull().sum()

<b>MOR HIT RATE</b> - total number of known times the donor has responded to a mailed solicitation from a group other than the charitable organization

In [ ]:
df.mor_hit_rate.isnull().sum()

In [ ]:
sns.distplot(df.mor_hit_rate[df.mor_hit_rate<10])

<b>NUMBER PROM 12</b> - number of promotions (card or other) sent to the individual by the charitable organization in the past 12 months

In [ ]:
sns.distplot(df.number_prom_12)

In [ ]:
df.number_prom_12.describe()

In [ ]:
df.number_prom_12.isnull().sum()

<b>OVERLAY_SOURCE</b> - the data source against which the individual was matched: M if Metromail, P if Polk, B if both

In [ ]:
df.overlay_source.value_counts()

In [ ]:
# df.overlay_source = df.overlay_source.map( {'M':0,'P':1,'B':2 } ).astype('int64')


<b>PCT_ATTRIBUTE1</b> - percent of residents in the neighborhood in which the individual lives that are males and active military

In [ ]:
df.pct_attribute1.isnull().sum()

In [ ]:
sns.distplot(df.pct_attribute1[df.pct_attribute1<10])

<b>PCT_ATTRIBUTE2</b> - percent of residents in the neighborhood in which the individual lives that are males and veterans

In [ ]:
df.pct_attribute2.plot(kind = 'hist')

In [ ]:
df.pct_attribute2.isnull().sum()

<b>PCT_ATTRIBUTE3</b> - percent of residents in the neighborhood in which the individual lives that are Vietnam veterans

In [ ]:
df.pct_attribute3.plot(kind = 'hist')

In [ ]:
df.pct_attribute3.isnull().sum()

<b>PCT_ATTRIBUTE4</b> - percent of residents in the neighborhood in which the individual lives that are WWII veterans

In [ ]:
df.pct_attribute4.plot(kind = 'hist')

In [ ]:
df.pct_attribute4.isnull().sum()

<b>PCT_OWNER_OCCUPIED</b> - percent of owner-occupied housing in the neighborhood in which the individual lives

In [ ]:
df.pct_owner_occupied.plot(kind = 'hist')

In [ ]:
df.pct_owner_occupied.isnull().sum()

PCT data is clean 

#### Pep star 

it's a binary variable, with no nulls, so everything is fine

#### Per Capita Income 

In [ ]:
sns.distplot(df.per_capita_income)

It was found during the exploration of median_house_value that there are some 0 values that are probably errors in the data. Here we follow the same strategy replacing those values with the average by urbanicity and SES

In [ ]:
#set the 0 to nan
df.loc[df.per_capita_income == 0,'per_capita_income']=np.NaN
#group by urbanicity and ses, calculate average and use that
df['per_capita_income'] = df.groupby(['ses','urbanicity'], sort=False)['per_capita_income'].apply(lambda x: x.fillna(x.mean()))

#### Published phone

it's a binary variable, with no nulls.

#### Recency status 96nk
description: recency status as of two years ago: A if active donor, S if star donor, N if new donor, E if inactive donor, F if first time donor, L if lapsing donor

In [ ]:
df.recency_status_96nk.value_counts()

Label encoding was tried but yielded worse results

#### RECENT_AVG_CARD_GIFT_AMT
average donation from the individual in response to a card solicitation from the charitable organization since four years ago

In [ ]:
sns.distplot( df['recent_avg_card_gift_amt'])

Data is fine

#### RECENT_AVG_GIFT_AMT
average donation (in \\$) from the individual to the charitable organization since four years ago


In [ ]:
sns.distplot(df.recent_avg_gift_amt)

Data looks fine!

#### RECENT_CARD_RESPONSE_COUNT
number of times the individual has responded to a card solicitation from the charitable organization since four years ago

In [ ]:
sns.distplot(df.recent_card_response_count)

This is fine

#### RECENT_CARD_RESPONSE_PROP
proportion of responses to the individual to the number of card solicitations from the charitable organization since four years ago


In [ ]:
sns.distplot(df.recent_card_response_prop)

**note:** ideas to try
* bin variable
* could be done in categories as most is under 0.6


#### RECENT_RESPONSE_COUNT
number of times the individual has responded to a promotion (card or other) from the charitable organization since four years ago


In [ ]:
sns.distplot(df.recent_response_count)

**note:** ideas to try
* bin variable
* bins 0,1,2,3,4,5,6,>7?



#### RECENT_RESPONSE_PROP
proportion of responses to the individual to the number of (card or other) solicitations from the charitable organization since four years ago


In [ ]:
sns.distplot(df.recent_response_prop)

Data looks fine

#### RECENT_STAR_STATUS
1 if individual has achieved star donor status since four years ago, 0 if not


In [ ]:
print(df.recent_star_status.value_counts())

What to do with values above 1. Let's see whether these people have donated and whether they look like people with star status 1. Probably need clustering to check this.

In [ ]:
df[df.recent_star_status > 0].sort_values(by = ['lifetime_gift_amount'])

**What to do with all the values > 1?**

Replace with 1: When looking at the lifetime_gift_amount, they have donated a lot, so it is probable that they should have received star status. Therefore, imputing with 1



In [ ]:
df['recent_star_status'].values[df.recent_star_status > 1] = 1
df.recent_star_status.value_counts()

#### SES
one of 5 possible socioeconomic codes classifying the neighborhood in which the individual lives

In [ ]:
df_reset()
df.ses.value_counts()

The weird '?' value correspond to the weird '.' value in the cluster code feature. Therefore, these are already taken out by that error removal.

#### URBANICITY
classification of the neighborhood in which the individual lives: U if urban, C if city, S if suburban, T if town, R if rural, ? if missing

In [ ]:
df.urbanicity.value_counts()

Same as for SES, '?' values are simply removed

#### WEALTH_RATING
one of 10 possible wealth rating groups based on a number of demographic characteristics

In [ ]:
sns.distplot(df.wealth_rating)

In [ ]:
df.wealth_rating.value_counts()

Let's check correlations

In [ ]:
df_label_encoded.corr().abs()['wealth_rating'].sort_values(ascending=False)[:10]

In [ ]:
sns.barplot(data=df,x="wealth_rating",y="median_household_income")

the same strategy as in income_group will be adopted

In [ ]:
#binning
df['median_household_bin'] = pd.qcut(df.median_household_income,9,labels=False)
#imputation
df['wealth_rating'] = df.groupby(['median_household_bin'], sort=False)['wealth_rating'].apply(lambda x: x.fillna(x.mean())).copy()
df['wealth_rating'] = df['wealth_rating'].round()
df.drop('median_household_bin',axis=1,inplace=True)

to help with the rest of the EDA we'll use the pandas profilling tool:

In [ ]:
#import data and drop duplicates or irrelevant
raw_df = pd.read_csv("Donors_dataset.csv")
df = raw_df.drop(["FILE_AVG_GIFT","CONTROL_NUMBER"],axis = 1)

#reorder df columns in alphabetical order
ordered_names = list(df.columns)
ordered_names.sort()
df = df[ordered_names]
#column names to lowercase
df.columns = df.columns.str.lower()

In [ ]:
#run the following to install pandas-proffiling conda install -c conda-forge pandas-profiling
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="df Report",explorative=True)

profile.to_widgets()